In [ ]:
import requests
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib import patches
from io import BytesIO, StringIO
import os
import numpy as np
import cv2
import PIL
import cv2
from matplotlib.pyplot import imshow
from matplotlib.pyplot import rcParams
from matplotlib.patches import Polygon
from matplotlib.pyplot import gcf, gca
import threading

%matplotlib inline

In [ ]:
cap = cv2.VideoCapture(0)
#set the width and height, and UNSUCCESSFULLY set the exposure time
cap.set(3,1080)
cap.set(4,1024)
cap.set(15, 0.1)

ENDPOINT = 'https://westeurope.api.cognitive.microsoft.com/face/v1.0/detect'
#KEY = '2d0523e810c24bd5b7fd4448fbf71c67'
KEY = '5d4e91c5581544229d0cdc2bc73d89e1'


params = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'age,gender,emotion'
}

headers = {
    'Content-Type': 'application/octet-stream',
    'Ocp-Apim-Subscription-Key': KEY
}

    
while True:
    ret, img = cap.read()
    #print("[INFO]... IMG Type: ", type(img))
    #print("[INFO]... IMAGE: ", img)
    
    f = BytesIO()
    PIL.Image.fromarray(img).save(f, 'png')
    data = f.getvalue()
    
    response = requests.post(data=data,url=ENDPOINT,headers=headers,params=params)
    
    rcParams['figure.figsize'] = (24, 16)
    ax = imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    for face in response.json():

        rect = face['faceRectangle']
        fattr = face['faceAttributes']

        left, top, width, height = rect['left'], rect['top'], rect['width'], rect['height']

        path = [[left, top], [left + width, top], 
                [left + width, top + height], [left, top + height]]

        ax.axes.add_patch(Polygon(path, edgecolor='red', facecolor='none'))

        disp = {
                'gender': fattr['gender'],
                'age': fattr['age']
               }

        disp.update(fattr['emotion'])

        print(disp)
        for i, k in enumerate(disp):
            ax.axes.text(left+width+5, top + 10 + 30*i, "{0}: {1}".format(k, disp[k]),
                         color='lime', fontsize=25, backgroundcolor='black')
    
    
    imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    key = cv2.waitKey(1)
    if key == 27:
        break


cv2.destroyAllWindows() 
cv2.VideoCapture(0).release()